### Exercício 2 – Contagem de palavras
- Crie uma lista com algumas frases (ex.: 3 ou 4 frases curtas).
- Transforme essa lista em um RDD ou DataFrame.
- Faça um processamento para contar quantas vezes cada palavra aparece.
- Mostre o resultado ordenado pelas palavras em ordem alfabética.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode 

In [2]:
spark = SparkSession.builder \
    .appName('WordCount') \
    .master("spark://spark:7077") \
    .config("spark.ui.enabled", "true") \
    .getOrCreate()

#### 1 - Criar o df com as frases

In [3]:
data = [
    ('Hello World',),
    ('Hello Spark',),
    ('Hello PySpark World',),
    ('Hello Fran',)
]

df = spark.createDataFrame(data, ['frase'])

print(f'Frases iniciais: ')
df.show(truncate=False)

Py4JJavaError: An error occurred while calling o27.sessionState.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:92)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:75)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:143)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:142)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at java.base/jdk.internal.reflect.Native

#### 2 - Quebrar cada frase em palavras
A função split(col, " ") divide uma string em uma lista (array), usando o espaço como separador.
##### Exemplo:

"Hello world" → ["Hello", "World"]

In [ ]:
df_split = df.withColumn('palavras', split(df['frase'], ' '))

#### 3 - Explodir o array em várias linhas
explode(array) “explode” cada elemento do array em uma linha nova.
##### Exemplo: ["Hello", "World"] → duas linhas:

Hello

World

In [ ]:
df_explode = df_split.withColumn('palavra', explode(df_split['palavras']))

#### 4 - Agrupar e contar as palavras
groupBy("palavra") → agrupa todas as linhas pela palavra.

.count() → conta quantas vezes cada palavra apareceu.

.orderBy("palavra") → ordena alfabeticamente.

In [ ]:
df_count = df_explode.groupBy('palavra').count().orderBy('palavra')

print('Contagem de palavras: ')

df_count.show()

In [ ]:
#spark.stop()